In [2]:
from utils import *
from data import *
from extraction import *
import math

In [4]:
structure = Structure.LVM
x_test = []
y_test = []
label_to_number = {label: number for number, label in enumerate(CLASS_LABELS)}
for i in range(1, PT_NUM+1):
    mask_4d = load_mask(i)
    dia_mask = img_mask_extraction(i, Frame.END_DIASTOLIC, CROP_SIZE)
    sys_mask = img_mask_extraction(i, Frame.END_SYSTOLIC, CROP_SIZE)
    dyn_dict = get_dynamic_features(mask_4d, structure)
    dia_dict = get_features(dia_mask, structure)
    sys_dict = get_features(sys_mask, structure)
    static_dict = {}
    for key, value in dia_dict.items():
        match key:
            case "mean_circularity":
                static_dict["mean_circularity"] = np.mean([value, sys_dict[key]])
            case "max_circumference":
                static_dict["max_circumference"] = np.max([value, sys_dict[key]])
            case "mean_circumference":
                static_dict["mean_circumference"] = np.mean([value, sys_dict[key]])
        if structure == Structure.LVM:
            match key:
                case "max_thickness":
                    static_dict["max_thickness"] = np.max([value, sys_dict[key]])
                case "min_thickness":
                    static_dict["min_thickness"] = np.min([value, sys_dict[key]])
                case "std_thickness":
                    static_dict["std_thickness"] = math.sqrt((value**2 + sys_dict[key]**2) / 2)
                case "mean_thickness":
                    static_dict["mean_thickness"] = np.mean([value, sys_dict[key]])
    feature_dict = {**dyn_dict, **static_dict}
    df = training_data_DF[['Group', 'Height', 'Weight']].loc[i-1]
    print(df)
    feature_dict['bmi'] = bmi(float(df['Height']), float(df['Weight']))
    feature_dict['body_surface_area'] = mosteller_method(float(df['Height']), float(df['Weight']))
    print(feature_dict)
    features = np.array(list(feature_dict.values()))
    x_test.append(features)
    y_test.append(label_to_number[str(df['Group'])])

Group       DCM
Height    184.0
Weight     95.0
Name: 0, dtype: object


/home/ethan/IDC_Group1/extraction.py:104: RuntimeWarning: invalid value encountered in divide
  ejection_fraction = (stroke_volume / end_diastolic_volume) * 100
/home/ethan/IDC_Group1/extraction.py:185: RuntimeWarning: invalid value encountered in scalar divide
  feature_dict['ratio_min_lv_rv'] = lv_volumes.min() / rv_volumes.min()
/home/ethan/IDC_Group1/extraction.py:186: RuntimeWarning: invalid value encountered in scalar divide
  feature_dict['ratio_min_lm_lv'] = lv_mass.min() / lv_volumes.min()
/home/ethan/IDC_Group1/extraction.py:187: RuntimeWarning: invalid value encountered in scalar divide
  feature_dict['ratio_min_rv_lm'] = rv_volumes.min() / lv_mass.min()


TypeError: unsupported operand type(s) for /: 'str' and 'int'